In [1]:
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [3]:
!pip install imblearn

In [4]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [6]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [7]:
!pip install lightgbm

In [8]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier

In [9]:
pip install warnings-plugin

Note: you may need to restart the kernel to use updated packages.


In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
from warnings import simplefilter 
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

## Loading the dataset

In [12]:
df = pd.read_csv("online_shoppers_intention.csv")

In [13]:
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [14]:
print(df.shape)

(12330, 18)


In [15]:
df.columns

Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       'Weekend', 'Revenue'],
      dtype='object')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

In [17]:
df.isnull().sum()

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64

In [18]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Administrative,12330.0,2.315166,3.321784,0.0,0.000000,1.000000,4.000000,27.000000
Administrative_Duration,12330.0,80.818611,176.779107,0.0,0.000000,7.500000,93.256250,3398.750000
Informational,12330.0,0.503569,1.270156,0.0,0.000000,0.000000,0.000000,24.000000
Informational_Duration,12330.0,34.472398,140.749294,0.0,0.000000,0.000000,0.000000,2549.375000
ProductRelated,12330.0,31.731468,44.475503,0.0,7.000000,18.000000,38.000000,705.000000
ProductRelated_Duration,12330.0,1194.746220,1913.669288,0.0,184.137500,598.936905,1464.157214,63973.522230
BounceRates,12330.0,0.022191,0.048488,0.0,0.000000,0.003112,0.016813,0.200000
ExitRates,12330.0,0.043073,0.048597,0.0,0.014286,0.025156,0.050000,0.200000
PageValues,12330.0,5.889258,18.568437,0.0,0.000000,0.000000,0.000000,361.763742
SpecialDay,12330.0,0.061427,0.198917,0.0,0.000000,0.000000,0.000000,1.000000


## Feature Engineering

In [19]:
df['Weekend'] = df['Weekend'].replace((True, False), (1,0))
df['Revenue'] = df['Revenue'].replace((True, False),(1,0))

In [20]:
df['Revenue'].value_counts()

0    10422
1     1908
Name: Revenue, dtype: int64

In [21]:
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,0,0
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,0,0
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,0,0
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,0,0
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,1,0


In [22]:
# VisitorType contains either Returning_Visitor or New_Visitor
# Let’s add Returning_Visitor column to existing dataframe    

In [23]:
df['VisitorType'].value_counts()

Returning_Visitor    10551
New_Visitor           1694
Other                   85
Name: VisitorType, dtype: int64

In [24]:
condition = df['VisitorType'] == 'Returning_Visitor'
df['Returning_Visitor'] = np.where(condition, 1, 0)

df = df.drop(columns=['VisitorType'])

In [25]:
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,Weekend,Revenue,Returning_Visitor
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,0,0,1
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,0,0,1
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,0,0,1
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,0,0,1
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,1,0,1


In [26]:
df['Returning_Visitor'].value_counts()

1    10551
0     1779
Name: Returning_Visitor, dtype: int64

## Applying Ordinal Encoding on Month column

In [27]:
df['Month'].value_counts()

May     3364
Nov     2998
Mar     1907
Dec     1727
Oct      549
Sep      448
Aug      433
Jul      432
June     288
Feb      184
Name: Month, dtype: int64

In [28]:
df['Month'].unique()

array(['Feb', 'Mar', 'May', 'Oct', 'June', 'Jul', 'Aug', 'Nov', 'Sep',
       'Dec'], dtype=object)

In [29]:
ordinal_encoder = OrdinalEncoder()
df['Month'] = ordinal_encoder.fit_transform(df[['Month']])

In [30]:
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,Weekend,Revenue,Returning_Visitor
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2.0,1,1,1,1,0,0,1
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,2.0,2,2,1,2,0,0,1
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,2.0,4,1,9,3,0,0,1
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,2.0,3,2,2,4,0,0,1
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,2.0,3,3,1,4,1,0,1


In [31]:
df['Month'].value_counts()

6.0    3364
7.0    2998
5.0    1907
1.0    1727
8.0     549
9.0     448
0.0     433
3.0     432
4.0     288
2.0     184
Name: Month, dtype: int64

## Checking correlation on revenue column

In [32]:
result = df[df.columns[1:]].corr()['Revenue']
result1 = result.sort_values(ascending=False)

In [33]:
result1

Revenue                    1.000000
PageValues                 0.492569
ProductRelated             0.158538
ProductRelated_Duration    0.152373
Informational              0.095200
Administrative_Duration    0.093587
Month                      0.080150
Informational_Duration     0.070345
Weekend                    0.029295
Browser                    0.023984
TrafficType               -0.005113
Region                    -0.011595
OperatingSystems          -0.014668
SpecialDay                -0.082305
Returning_Visitor         -0.103843
BounceRates               -0.150673
ExitRates                 -0.207071
Name: Revenue, dtype: float64

## Prepairing Features as X and target as y

In [34]:
X = df.drop(['Revenue'], axis=1)
y = df['Revenue']

## Train and Test split

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

## Model Pipeline

In [36]:
def model_pipeline(X, model):
    n_c = X.select_dtypes(exclude=['object']).columns.values.tolist()
    c_c = X.select_dtypes(include=['object']).columns.values.tolist()
    
    numeric_columns = n_c
    categorical_columns = c_c
    
    numeric_pipeline = SimpleImputer(strategy = 'constant')
    categorical_pipeline = OneHotEncoder(handle_unknown = 'ignore')
    
    a = ('numeric', numeric_pipeline, numeric_columns)
    b = ('categorical', categorical_pipeline, categorical_columns)
    
    preprocessor = ColumnTransformer(
    transformers = [a,b],
    remainder = 'passthrough'
    )
    
    c = ('preprocessor', preprocessor)
    d = ('smote', SMOTE(random_state=1))
    e = ('scaler', MinMaxScaler())
    f = ('feature_selection', SelectKBest(score_func = chi2, k=6))
    g = ('model', model)
    
    bundled_pipeline = imbpipeline(steps = [c,d,e,f,g])
    
    return bundled_pipeline

## Model Selection

In [37]:
def select_model(X,y,pipeline=None):
    
    classifiers = {}
    
    c_d1 = {'DummyClassifier': DummyClassifier(strategy='most_frequent')}
    classifiers.update(c_d1)
    
    xgb = XGBClassifier(
        verbosity = 0,
        use_label_encoder = False,
        eval_metric = 'logloss',
        objective = 'binary:logistic',
    )
    
    c_d2 = {'XGBClassifier': xgb}
    classifiers.update(c_d2)
    
    c_d3 = {'LGBMClassifier': LGBMClassifier()}
    classifiers.update(c_d3)
    
    c_d4 = {'RandomForesClassifier': RandomForestClassifier()}
    classifiers.update(c_d4)
    
    c_d5 = {'DecisionTreeClassifier': DecisionTreeClassifier()}
    classifiers.update(c_d5)
    
    c_d6 = {'ExtraTreeClassifier': ExtraTreeClassifier()}
    classifiers.update(c_d6)
    
    c_d7 = {'ExtraTreesClassifier': ExtraTreeClassifier()}
    classifiers.update(c_d7)
    
    c_d8 = {'AdaBoostClassifier': AdaBoostClassifier()}
    classifiers.update(c_d8)
    
    c_d9 = {'KNeighborsClassifier': KNeighborsClassifier()}
    classifiers.update(c_d9)
    
    c_d10 = {"RidgeClassifier": RidgeClassifier()}
    classifiers.update(c_d10)

    c_d11 = {"SGDClassifier": SGDClassifier()}
    classifiers.update(c_d11)

    c_d12 = {"BaggingClassifier": BaggingClassifier()}
    classifiers.update(c_d12)

    c_d13 = {"BernoulliNB": BernoulliNB()}
    classifiers.update(c_d13)

    c_d14 = {"SVC": SVC()}
    classifiers.update(c_d14)

    c_d15 = {"MLPClassifier": MLPClassifier()}
    classifiers.update(c_d15)
    
    mlpc = {
        "MLPClassifier (paper)": MLPClassifier(hidden_layer_sizes=(27,50),
                                              max_iter = 300,
                                              activation = 'relu',
                                              solver = 'adam',
                                              random_state=1)
    }
    c_d16 = mlpc
    classifiers.update(c_d16)
    
    cols = ['model','run_time','roc_auc']
    df_models = pd.DataFrame(columns=cols)
    
    for key in classifiers:
        
        start_time = time.time()
        
        print()
        print('model_pipeline run succesfully on', key)
        
        pipeline = model_pipeline(X_train, classifiers[key])
        
        cv = cross_val_score(pipeline,X,y,cv=10, scoring = 'roc_auc')
        
        row = {'model': key,
              'run_time':format(round((time.time()-start_time)/60,2)),
              'roc_auc': cv.mean(),
              }
        
        df_models = pd.concat([df_models, pd.DataFrame([row])], ignore_index=True)
        
    df_models = df_models.sort_values(by='roc_auc', ascending=False)
    return df_models
    

## Access Model - select_model function

In [38]:
models = select_model(X_train,y_train)


model_pipeline run succesfully on DummyClassifier

model_pipeline run succesfully on XGBClassifier

model_pipeline run succesfully on LGBMClassifier

model_pipeline run succesfully on RandomForesClassifier

model_pipeline run succesfully on DecisionTreeClassifier

model_pipeline run succesfully on ExtraTreeClassifier

model_pipeline run succesfully on ExtraTreesClassifier

model_pipeline run succesfully on AdaBoostClassifier

model_pipeline run succesfully on KNeighborsClassifier

model_pipeline run succesfully on RidgeClassifier

model_pipeline run succesfully on SGDClassifier

model_pipeline run succesfully on BaggingClassifier

model_pipeline run succesfully on BernoulliNB

model_pipeline run succesfully on SVC

model_pipeline run succesfully on MLPClassifier

model_pipeline run succesfully on MLPClassifier (paper)


In [39]:
models

,model,run_time,roc_auc
14,MLPClassifier,2.72,0.9033
15,MLPClassifier (paper),4.22,0.900118
2,LGBMClassifier,0.09,0.897217
1,XGBClassifier,0.16,0.891174
7,AdaBoostClassifier,0.07,0.888264
10,SGDClassifier,0.01,0.887928
13,SVC,2.51,0.885963
3,RandomForesClassifier,0.18,0.885935
11,BaggingClassifier,0.05,0.864238
12,BernoulliNB,0.01,0.857851


## Accessing best model and trainig

In [40]:
selected_model = MLPClassifier()
bundled_pipeline = model_pipeline(X_train, selected_model)
bundled_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric',
                                                  SimpleImputer(strategy='constant'),
                                                  ['Administrative',
                                                   'Administrative_Duration',
                                                   'Informational',
                                                   'Informational_Duration',
                                                   'ProductRelated',
                                                   'ProductRelated_Duration',
                                                   'BounceRates', 'ExitRates',
                                                   'PageValues', 'SpecialDay',
                                                   'Month', 'OperatingSystems',
                                                   'Browser', 'Region',
           

In [41]:
y_pred = bundled_pipeline.predict(X_test)

In [42]:
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

## ROC and AUC Score

In [43]:
roc_auc = roc_auc_score(y_test,y_pred)
accuracy = accuracy_score(y_test,y_pred)
f1_score = f1_score(y_test,y_pred)

In [44]:
roc_auc

0.8341870030419658

In [45]:
accuracy

0.8767234387672344

In [46]:
f1_score

0.6775106082036776

In [47]:
## Classification Report

In [48]:
classification_report = classification_report(y_test,y_pred)

In [49]:
print(classification_report)

              precision    recall  f1-score   support

           0       0.95      0.90      0.92      3077
           1       0.60      0.77      0.68       622

    accuracy                           0.88      3699
   macro avg       0.78      0.83      0.80      3699
weighted avg       0.89      0.88      0.88      3699

